## 1. 形参设定

In [2]:
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
parser = ArgumentParser("BiNE",
                        formatter_class=ArgumentDefaultsHelpFormatter,
                        conflict_handler='resolve')

parser.add_argument('--train-data', default=r'../data/rating_train.dat',
                    help='Input graph file.')

parser.add_argument('--test-data', default=r'../data/rating_test.dat')

parser.add_argument('--model-name', default='default',
                    help='name of model.')

parser.add_argument('--vectors-u', default=r'../data/vectors_u.dat',
                    help="file of embedding vectors of U")

parser.add_argument('--vectors-v', default=r'../data/vectors_v.dat',
                    help="file of embedding vectors of V")

parser.add_argument('--case-train', default=r'../data/wiki/case_train.dat',
                    help="file of training data for LR")

parser.add_argument('--case-test', default=r'../data/wiki/case_test.dat',
                    help="file of testing data for LR")

parser.add_argument('--ws', default=5, type=int,
                    help='window size.')

parser.add_argument('--ns', default=4, type=int,
                    help='number of negative samples.')

parser.add_argument('--d', default=128, type=int,
                    help='embedding size.')

parser.add_argument('--maxT', default=32, type=int,
                    help='maximal walks per vertex.')

parser.add_argument('--minT', default=1, type=int,
                    help='minimal walks per vertex.')

parser.add_argument('--p', default=0.15, type=float,
                    help='walk stopping probability.')

parser.add_argument('--alpha', default=0.01, type=float,
                    help='trade-off parameter alpha.')

parser.add_argument('--beta', default=0.01, type=float,
                    help='trade-off parameter beta.')

parser.add_argument('--gamma', default=0.1, type=float,
                    help='trade-off parameter gamma.')

parser.add_argument('--lam', default=0.01, type=float,
                    help='learning rate lambda.')
parser.add_argument('--max-iter', default=50, type=int,
                    help='maximal number of iterations.')

parser.add_argument('--top-n', default=10, type=int,
                    help='recommend top-n items for each user.')

parser.add_argument('--rec', default=0, type=int,
                    help='calculate the recommendation metrics.')

parser.add_argument('--lip', default=1, type=int,
                    help='calculate the link prediction metrics.')

parser.add_argument('--large', default=0, type=int,
                    help='for large bipartite, 1 do not generate homogeneous graph file; 2 do not generate homogeneous graph')

parser.add_argument('--mode', default='hits', type=str,
                    help='metrics of centrality')

args, _ = parser.parse_known_args()
args.train_data='../data/wiki/rating_train.dat'
args.test_data='../data/wiki/ratindg_test.dat'
args.vectors_u='../data/wiki/vectors_u.dat'
args.vectors_v='../data/wiki/vectors_v.dat'
args

Namespace(train_data='../data/wiki/rating_train.dat', test_data='../data/wiki/ratindg_test.dat', model_name='default', vectors_u='../data/wiki/vectors_u.dat', vectors_v='../data/wiki/vectors_v.dat', case_train='../data/wiki/case_train.dat', case_test='../data/wiki/case_test.dat', ws=5, ns=4, d=128, maxT=32, minT=1, p=0.15, alpha=0.01, beta=0.01, gamma=0.1, lam=0.01, max_iter=50, top_n=10, rec=0, lip=1, large=0, mode='hits')

In [3]:
import os
model_path = os.path.join('../', args.model_name)
if os.path.exists(model_path) is False:
    os.makedirs(model_path)
print(model_path)

../default


## 2. 实验参数设定

In [4]:
alpha, beta, gamma, lam = args.alpha, args.beta, args.gamma, args.lam
print('======== experiment settings =========')
print('alpha : %0.4f, beta : %0.4f, gamma : %0.4f, lam : %0.4f, p : %0.4f, ws : %d, ns : %d, maxT : % d, minT : %d, max_iter : %d, d : %d' % (alpha, beta, gamma, lam, args.p, args.ws, args.ns,args.maxT,args.minT,args.max_iter, args.d))

======== experiment settings =========
alpha : 0.0100, beta : 0.0100, gamma : 0.1000, lam : 0.0100, p : 0.1500, ws : 5, ns : 4, maxT :  32, minT : 1, max_iter : 50, d : 128


## 3. 数据加载

In [5]:
class DataUtils(object):
    def __init__(self, model_path):
        self.model_path = model_path

    def rename(self, datafile):
        """
        Distinguish two types of node and rename
        """
        with open(os.path.join(self.model_path,"_ratings.dat"), "w") as fw:
            with open(datafile, "r", encoding="UTF-8") as fin:
                line = fin.readline()
                while line:
                    user, item, rating = line.strip().split("\t")
                    fw.write("u" + user + "\t" + "i" + item + "\t" + rating + "\n")
                    line = fin.readline()
                    
## 拆分数据形成训练集和测试集
    def split_data(self, percent):
        """
        split data
        :param percent:
        :return:
        """
        ##分为两个元组和两个字典，元组储存两种不同类别得节点，字典储存的是边连接和其权重
        test_user,test_item,test_rate,rating = set(), set(), {},{}
        with open(os.path.join(self.model_path, "ratings.dat"), "r") as fin, open(os.path.join(self.model_path, "ratings_train.dat"),"w") as ftrain, open(os.path.join(self.model_path,"ratings_test.dat"), "w") as ftest:
            ##读取ratings文件的每行
            for line in fin.readlines():
                ##将每行分割为用户、项目和边权重，然后存储在 rating 字典中
                user, item, rate = line.strip().split("\t")
                if rating.get(user) is None:
                    rating[user] = {}
                rating[user][item] = rate
            ##对于 rating 字典中的每个用户，随机选取一定比例的项目作为测试集，其余的作为训练集
            for u in rating.keys():
                item_list = rating[u].keys()
                sample_list = random.sample(item_list, int(len(item_list) * percent))
                ##将选定的测试集数据写入 ratings_train.dat，将剩余的数据写入 ratings_test.dat。
                for item in item_list:
                    if item in sample_list:
                        ftrain.write(u + "\t" + item + "\t" + rating[u][item] + "\n")
                    else:
            ##对于测试集中的每个边权重，将其添加到 test_rate 字典中，并记录相应的用户和项目。
                        if test_rate.get(u) is None:
                            test_rate[u] = {}
                        test_rate[u][item] = float(rating[u][item])
                        test_user.add(u)
                        test_item.add(item)
                        ftest.write(u + "\t" + item + "\t" + rating[u][item] + "\n")
        return test_user, test_item, test_rate
    
## 读取数据，得到两种节点和之间的边权重
    def read_data(self,filename=None):
        if filename is None:
            filename = os.path.join(self.model_path,"ratings_test.dat")
        users,items,rates = set(), set(), {}
        with open(filename, "r", encoding="UTF-8") as fin:
            line = fin.readline()
            while line:
                user, item, rate = line.strip().split()
                if rates.get(user) is None:
                    rates[user] = {}
                rates[user][item] = float(rate)
                users.add(user)
                items.add(item)
                line = fin.readline()
        return users, items, rates


In [6]:
print('========== processing data ===========')
dul = DataUtils(model_path)
if args.rec:
    test_user, test_item, test_rate = dul.read_data(args.test_data)

========== processing data ===========


## 4.图加载

In [7]:
import networkx as nx
import graph
import random
from networkx.algorithms import bipartite as bi
import numpy as np
from lsh import get_negs_by_lsh
from io import open
import os
import itertools

class GraphUtils(object):
    def __init__(self, model_path):
        self.model_path = model_path
        self.G = nx.Graph()
        self.edge_dict_u = {}
        self.edge_dict_v = {}
        self.edge_list = []
        self.node_u = []
        self.node_v = []
        self.authority_u, self.authority_v = {}, {}
        self.walks_u, self.walks_v = [], []
        self.G_u, self.G_v = None, None
        self.fw_u = os.path.join(self.model_path, "homogeneous_u.dat")
        self.fw_v = os.path.join(self.model_path, "homogeneous_v.dat")
        self.negs_u = {}
        self.negs_v = {}
        self.context_u = {}
        self.context_v = {}
## 4-1 建立训练用图这里通过之间我的预设即传入的文件是args.train_data='../data/wiki/rating_train.dat'
    def construct_training_graph(self, filename=None):
        if filename is None:
            filename = os.path.join(self.model_path, "ratings_train.dat")
        edge_list_u_v = []
        edge_list_v_u = []
        ##遍历训练train_data，
        with open(filename, encoding="UTF-8") as fin:
            line = fin.readline()
            while line:
                user, item, rating = line.strip().split("\t")
                ##检查并初始化用户和项目的边字典。
                if self.edge_dict_u.get(user) is None:
                    self.edge_dict_u[user] = {}
                if self.edge_dict_v.get(item) is None:
                    self.edge_dict_v[item] = {}
               ##用户-项目和项目-用户的边添加到对应的列表中，并更新边字典
                edge_list_u_v.append((user, item, float(rating)))
                self.edge_dict_u[user][item] = float(rating)
                self.edge_dict_v[item][user] = float(rating)
                edge_list_v_u.append((item, user, float(rating)))
                line = fin.readline()
        
        #建立二部图
        self.node_u = self.edge_dict_u.keys()
        self.node_v = self.edge_dict_v.keys()
        # self.node_u.sort()
        # self.node_v.sort()
        ##将u和v节点标记为两个不同的部分
        self.G.add_nodes_from(self.node_u, bipartite=0)
        self.G.add_nodes_from(self.node_v, bipartite=1)
        self.G.add_weighted_edges_from(edge_list_u_v+edge_list_v_u)
        ##用户-项目的边列表存储在 self.edge_list
        self.edge_list = edge_list_u_v
## 5-1 计算节点中心性
    def calculate_centrality(self, mode='hits'):
        ##如果模式设定的是计算度中心性即以其节点的度来决定其重要性
        if mode == 'degree_centrality':
            a = nx.degree_centrality(self.G)
        ## 否则默认使用HITS计算图的权威值和枢纽值
        else:
            h, a = nx.hits(self.G)
        ##初始化最大权威值和最小权威值
        max_a_u, min_a_u,max_a_v,min_a_v = 0, 100000, 0, 100000
        ##遍历所有节点
        for node in self.G.nodes():
            ##对u节点和i节点进行更新节点类型的最大和最小权威值
            if node[0] == "u":
                if max_a_u < a[node]:
                    max_a_u = a[node]
                if min_a_u > a[node]:
                    min_a_u = a[node]
            if node[0] == "i":
                if max_a_v < a[node]:
                    max_a_v = a[node]
                if min_a_v > a[node]:
                    min_a_v = a[node]
        ##二次遍历
        for node in self.G.nodes():
            ##计算其权威值的归一化分数并储存在authority_u
            if node[0] == "u":
                ##如果最大值和最小值相同，即分母为零，则将归一化后的权威值设为0
                if max_a_u-min_a_u != 0:
                    self.authority_u[node] = (float(a[node])-min_a_u) / (max_a_u-min_a_u)
                else:
                    self.authority_u[node] = 0
            ####计算其权威值的归一化分数并储存在authority_v
            if node[0] == 'i':
                ####如果最大值和最小值相同，即分母为零，则将归一化后的权威值设为0
                if max_a_v-min_a_v != 0:
                    self.authority_v[node] = (float(a[node])-min_a_v) / (max_a_v-min_a_v)
                else:
                    self.authority_v[node] = 0
## 5-2 针对小图进行同构图随机游走
    def homogeneous_graph_random_walks(self, percentage, maxT, minT):
        ## 构建二部图的邻接矩阵
        A = bi.biadjacency_matrix(self.G, self.node_u, self.node_v, dtype=np.float64,weight='weight', format='csr')
        
        ## 建立索引映射，创造字典来映射行索引和列索引的节点并且进行反向索引，从而从矩阵的索引回到图中节点
        row_index = dict(zip(self.node_u, itertools.count()))
        col_index = dict(zip(self.node_v, itertools.count()))
        index_row = dict(zip(row_index.values(), row_index.keys()))
        index_item = dict(zip(col_index.values(), col_index.keys()))
        
        ##矩阵转置
        AT = A.transpose()
        
        ##保存同构图5-2-1 save_homogenous_graph_to_file
        self.save_homogenous_graph_to_file(A.dot(AT),self.fw_u, index_row,index_row)
        self.save_homogenous_graph_to_file(AT.dot(A),self.fw_v, index_item,index_item)
        
        ## 5-2-2 对u和v的同构图分别进行随机游走
        self.G_u, self.walks_u = self.get_random_walks_restart(self.fw_u, self.authority_u, percentage=percentage, maxT=maxT, minT=minT)
        self.G_v, self.walks_v = self.get_random_walks_restart(self.fw_v, self.authority_v, percentage=percentage, maxT=maxT, minT=minT)

## 5-2-2 重新开始随机游走
    def get_random_walks_restart(self, datafile, hits_dict, percentage, maxT, minT):
        if datafile is None:
            datafile = os.path.join(self.model_path,"rating_train.dat")
        ## 加载图 5-2-2-1 
        G = graph.load_edgelist(datafile, undirected=True)
        print("number of nodes: {}".format(len(G.nodes())))
        print("walking...")
        ##获取随机游走的语料 5-2-2-2
        walks = graph.build_deepwalk_corpus_random(G, hits_dict, percentage=percentage, maxT = maxT, minT = minT, alpha=0)
        print("walking...ok")
        return G, walks

    def homogeneous_graph_random_walks_for_large_bipartite_graph(self, percentage, maxT, minT):
        A = bi.biadjacency_matrix(self.G, self.node_u, self.node_v, dtype=np.float64,weight='weight', format='csr')
        row_index = dict(zip(self.node_u, itertools.count()))
        col_index = dict(zip(self.node_v, itertools.count()))
        index_row = dict(zip(row_index.values(), row_index.keys()))
        index_item = dict(zip(col_index.values(), col_index.keys()))
        AT = A.transpose()
        matrix_u = self.get_homogenous_graph(A.dot(AT), self.fw_u, index_row, index_row)
        matrix_v = self.get_homogenous_graph(AT.dot(A), self.fw_v, index_item, index_item)
        self.G_u, self.walks_u = self.get_random_walks_restart_for_large_bipartite_graph(matrix_u, self.authority_u, percentage=percentage, maxT=maxT, minT=minT)
        self.G_v, self.walks_v = self.get_random_walks_restart_for_large_bipartite_graph(matrix_v, self.authority_v, percentage=percentage, maxT=maxT, minT=minT)

    def homogeneous_graph_random_walks_for_large_bipartite_graph_without_generating(self, datafile, percentage, maxT, minT):
        self.G_u, self.walks_u = self.get_random_walks_restart_for_large_bipartite_graph_without_generating(datafile, self.authority_u, percentage=percentage, maxT=maxT, minT=minT, node_type='u')
        self.G_v, self.walks_v = self.get_random_walks_restart_for_large_bipartite_graph_without_generating(datafile, self.authority_v, percentage=percentage, maxT=maxT, minT=minT,node_type='i')

    def get_random_walks_restart_for_large_bipartite_graph(self, matrix, hits_dict, percentage, maxT, minT):
        G = graph.load_edgelist_from_matrix(matrix, undirected=True)
        print("number of nodes: {}".format(len(G.nodes())))
        print("walking...")
        walks = graph.build_deepwalk_corpus_random(G, hits_dict, percentage=percentage, maxT = maxT, minT = minT, alpha=0)
        print("walking...ok")
        return G, walks

    def get_random_walks_restart_for_large_bipartite_graph_without_generating(self, datafile, hits_dict, percentage, maxT, minT, node_type='u'):
        if datafile is None:
            datafile = os.path.join(self.model_path,"rating_train.dat")
        G = graph.load_edgelist(datafile, undirected=True)
        cnt = 0
        for n in G.nodes():
            if n[0] == node_type:
                cnt += 1
        print("number of nodes: {}".format(cnt))
        print("walking...")
        walks = graph.build_deepwalk_corpus_random_for_large_bibartite_graph(G, hits_dict, percentage=percentage, maxT = maxT, minT = minT, alpha=0,node_type=node_type)
        # print(walks)
        print("walking...ok")
        return G, walks


    def save_words_and_sentences_to_file(self, filenodes, filesentences):
        with open(filenodes,"w") as fw:
            for node in self.G.keys():
                fw.write(node+"\n")

        with open(filesentences,"w") as fs:
            for nodes in self.walks:
                for index in range(0,len(nodes)):
                    if index == len(nodes)-1:
                        fs.write(nodes[index]+"\n")
                    else:
                        fs.write(nodes[index]+" ")
## 6-1 获取负采样节点
    def get_negs(self,num_negs):
        ## 6-1-1 使用LSH方法生成负样本 get_negs_by_lsh
        self.negs_u, self.negs_v = get_negs_by_lsh(self.edge_dict_u,self.edge_dict_v,num_negs)
        # print(len(self.negs_u),len(self.negs_v))
        return self.negs_u, self.negs_v

    def get_context_and_fnegatives(self,G,walks,win_size,num_negs,table):
        # generate context and negatives
        if isinstance(G, graph.Graph):
            node_list = G.nodes()
        elif isinstance(G, list):
            node_list = G
        word2id = {}
        for i in range(len(node_list)):
            word2id[node_list[i]] = i + 1
        walk_list = walks
        print("context...")
        context_dict = {}
        new_neg_dict = {}
        for step in range(len(walk_list)):

            walk = walk_list[step % len(walk_list)]
            # print(walk)
            batch_labels = []
            # travel each walk
            for iter in range(len(walk)):
                start = max(0, iter - win_size)
                end = min(len(walk), iter + win_size + 1)
                # index: index in window
                if context_dict.get(walk[iter]) is None:
                    context_dict[walk[iter]] = []
                    new_neg_dict[walk[iter]] = []
                labels_list = []
                neg_sample = []
                for index in range(start, end):
                    labels_list.append(walk[index])
                while len(neg_sample) < num_negs:
                    sa = random.choice(range(len(node_list)))
                    if table[sa] in labels_list:
                        continue
                    neg_sample.append(table[sa])
                context_dict[walk[iter]].append(labels_list)
                new_neg_dict[walk[iter]].append(neg_sample)
            if len(batch_labels) == 0:
                continue
        print("context...ok")
        return context_dict, new_neg_dict
    
## 6-2 get_context_and_negatives
    def get_context_and_negatives(self,G,walks,win_size,num_negs,negs_dict):
        # generate context and negatives
    ## 创建节点到ID的映射
        if isinstance(G, graph.Graph):
            node_list = G.nodes()
        elif isinstance(G, list):
            node_list = G
        word2id = {}
        node_list=list(node_list)
        for i in range(len(node_list)):
            word2id[node_list[i]] = i + 1
        walk_list = walks
        print("context...")
        context_dict = {}
        new_neg_dict = {}
    ## 遍历随机游走
        for step in range(len(walk_list)):
            walk = walk_list[step % len(walk_list)]
            # print(walk)
            # travel each walk
        ##确定当前节点的上下文窗口范围
            for iter in range(len(walk)):
                start = max(0, iter - win_size)
                end = min(len(walk), iter + win_size + 1)
                # index: index in window
                if context_dict.get(walk[iter]) is None:
                    context_dict[walk[iter]] = []
                    new_neg_dict[walk[iter]] = []
                labels_list = []
                ##从负样本字典中获取当前节点的负样本
                negs = negs_dict[walk[iter]]
                ##遍历窗口内的节点，将它们添加到上下文列表 labels_list
                for index in range(start, end):
                    ##如果窗口内节点在负样本中就从负样本中移除这些负样本节点
                    if walk[index] in negs:
                        negs.remove(walk[index])
                    if walk[index] == walk[iter]:
                        continue
                    else:
                        labels_list.append(walk[index])
                neg_sample = random.sample(negs,min(num_negs,len(negs)))
                ## 将上下文和负样本添加到相应的字典中
                context_dict[walk[iter]].append(labels_list)
                new_neg_dict[walk[iter]].append(neg_sample)
        print("context...ok")
        return context_dict, new_neg_dict
        # with open(context_file,'w', encoding='utf-8') as fw1, open(neg_file,'w', encoding='utf-8') as fw2:
        #     for u in context_dict.keys():
        #         fw1.write(u+"\t")
        #         fw2.write(u+"\t")
        #         lens = len(context_dict[u])
        #         for i in range(lens):
        #             str1 = u','.join(context_dict[u][i])
        #             str2 = u','.join(neg_dict[u][i])
        #             if i != lens -1:
        #                 fw1.write(str1+"\t")
        #                 fw2.write(str2+"\t")
        #             else:
        #                 fw1.write(str1+"\n")
        #                 fw2.write(str2+"\n")
        # return context_dict, neg_dict
## 5-2-1 保存同构图
    def save_homogenous_graph_to_file(self, A, datafile, index_row, index_item):
        (M,N) = A.shape
        csr_dict = A.__dict__
        data = csr_dict.get("data")
        indptr = csr_dict.get("indptr")
        indices = csr_dict.get("indices")
        col_index = 0
        with open(datafile,'w') as fw:
            for row in range(M):
                for col in range(indptr[row],indptr[row+1]):
                    r = row
                    c = indices[col]
                    fw.write(index_row.get(r)+"\t"+index_item.get(c)+"\t"+str(data[col_index])+"\n")
                    col_index += 1

    def get_homogenous_graph(self, A, datafile, index_row, index_item):
        (M,N) = A.shape
        csr_dict = A.__dict__
        data = csr_dict.get("data")
        indptr = csr_dict.get("indptr")
        indices = csr_dict.get("indices")
        col_index = 0
        matrix = {}
        with open(datafile,'w') as fw:
            for row in range(M):
                for col in range(indptr[row],indptr[row+1]):
                    r = index_row.get(row)
                    c = index_item.get(indices[col])
                    if matrix.get(r) is None:
                        matrix[r] = []
                    matrix[r].append(c)
                    col_index += 1

        return matrix

    def read_sentences_and_homogeneous_graph(self, filesentences=None, datafile=None):
        G = graph.load_edgelist(datafile, undirected=True)
        walks = []
        with open(filesentences,"r") as fin:
            for line in fin.readlines():
                walk = line.strip().split(" ")
                walks.append(walk)
        return G, walks


In [10]:
print("constructing graph....")
gul = GraphUtils(model_path)
## 4-1 建立训练用图
gul.construct_training_graph(args.train_data)
## 获取u节点连接哪些i节点和所对应的连接的边的权重
edge_dict_u = gul.edge_dict_u
# print(edge_dict_u)
## 获取边连接关系以列表的形式表现
edge_list = gul.edge_list
# print(edge_list)

constructing graph....
[('u3', 'i6', 1.0), ('u3', 'i7', 1.0), ('u3', 'i8', 1.0), ('u3', 'i11', 1.0), ('u3', 'i13', 1.0), ('u3', 'i15', 1.0), ('u3', 'i16', 1.0), ('u3', 'i19', 1.0), ('u3', 'i20', 1.0), ('u3', 'i22', 1.0), ('u3', 'i24', 1.0), ('u3', 'i26', 1.0), ('u3', 'i27', 1.0), ('u3', 'i28', 1.0), ('u3', 'i5', 1.0), ('u3', 'i30', 1.0), ('u3', 'i31', 1.0), ('u3', 'i34', 1.0), ('u3', 'i36', 1.0), ('u3', 'i37', 1.0), ('u3', 'i38', 1.0), ('u3', 'i40', 1.0), ('u3', 'i41', 1.0), ('u3', 'i42', 1.0), ('u3', 'i4', 1.0), ('u3', 'i43', 1.0), ('u3', 'i44', 1.0), ('u3', 'i46', 1.0), ('u3', 'i50', 1.0), ('u3', 'i51', 1.0), ('u3', 'i52', 1.0), ('u3', 'i53', 1.0), ('u3', 'i55', 1.0), ('u3', 'i57', 1.0), ('u3', 'i58', 1.0), ('u3', 'i59', 1.0), ('u3', 'i60', 1.0), ('u3', 'i63', 1.0), ('u3', 'i65', 1.0), ('u3', 'i66', 1.0), ('u3', 'i67', 1.0), ('u3', 'i68', 1.0), ('u3', 'i73', 1.0), ('u3', 'i75', 1.0), ('u3', 'i76', 1.0), ('u3', 'i77', 1.0), ('u3', 'i79', 1.0), ('u4', 'i6', 1.0), ('u4', 'i66', 1.0), ('

## 5. 随机游走生成器

In [46]:
def walk_generator(gul,args):
    """
    walk generator
    :param gul:
    :param args:
    :return:
    """
    ## 5-1 计算节点的中心性为之后的节点进行随机游走的次数提供依据
    gul.calculate_centrality(args.mode)
    
    ## 5-2 开始随机游走，根据图大小选择不同的随机游走函数这里我们选择的是0
    if args.large == 0:
        gul.homogeneous_graph_random_walks(percentage=args.p, maxT=args.maxT, minT=args.minT)
    elif args.large == 1:
        gul.homogeneous_graph_random_walks_for_large_bipartite_graph(percentage=args.p, maxT=args.maxT, minT=args.minT)
    elif args.large == 2:
        gul.homogeneous_graph_random_walks_for_large_bipartite_graph_without_generating(datafile=args.train_data,percentage=args.p,maxT=args.maxT, minT=args.minT)
    return gul

In [47]:
walk_generator(gul,args)

number of nodes: 15000
walking...
walking...ok
number of nodes: 2529
walking...
walking...ok


## 6.获取正样本和负样本

In [48]:
from sklearn import preprocessing
def get_context_and_negative_samples(gul, args):
    """
    get context and negative samples offline
    :param gul:
    :param args:
    :return: context_dict_u, neg_dict_u, context_dict_v, neg_dict_v,gul.node_u,gul.node_v
    """
    if args.large == 0:
        ## 6-1 进行负采样
        neg_dict_u, neg_dict_v = gul.get_negs(args.ns)
        print("negative samples is ok.....")
        ## 6-2 得到正采样节点和负采样节点
        context_dict_u, neg_dict_u = gul.get_context_and_negatives(gul.G_u, gul.walks_u, args.ws, args.ns, neg_dict_u)
        context_dict_v, neg_dict_v = gul.get_context_and_negatives(gul.G_v, gul.walks_v, args.ws, args.ns, neg_dict_v)
    else:
        neg_dict_u, neg_dict_v = gul.get_negs(args.ns)
        # print len(gul.walks_u),len(gul.walks_u)
        print("negative samples is ok.....")
        context_dict_u, neg_dict_u = gul.get_context_and_negatives(gul.node_u, gul.walks_u, args.ws, args.ns, neg_dict_u)
        context_dict_v, neg_dict_v = gul.get_context_and_negatives(gul.node_v, gul.walks_v, args.ws, args.ns, neg_dict_v)

    return context_dict_u, neg_dict_u, context_dict_v, neg_dict_v,gul.node_u,gul.node_v

In [ ]:
print("getting context and negative samples....")
## 获取上下文节点和负采样节点
context_dict_u, neg_dict_u, context_dict_v, neg_dict_v, node_u, node_v = get_context_and_negative_samples(gul, args)

## 对节点embedding进行初始化

In [49]:
def init_embedding_vectors(node_u, node_v, node_list_u, node_list_v, args):
    """
    initialize embedding vectors
    :param node_u:
    :param node_v:
    :param node_list_u:
    :param node_list_v:
    :param args:
    :return:
    """
    # user
    for i in node_u:
        ##创建两个随机向量 vectors 和 help_vectors，其维度由 args.d 指定。通过上下文向量去帮助vectors去学习并调整其embedding表示
        vectors = np.random.random([1, args.d])
        help_vectors = np.random.random([1, args.d])
        ##在 node_list_u 字典中为节点 i 创建一个新字典，存储其嵌入向量和上下文向量
        node_list_u[i] = {}
        ## 归一化
        node_list_u[i]['embedding_vectors'] = preprocessing.normalize(vectors, norm='l2')
        node_list_u[i]['context_vectors'] = preprocessing.normalize(help_vectors, norm='l2')
    # item——同理
    for i in node_v:
        vectors = np.random.random([1, args.d])
        help_vectors = np.random.random([1, args.d])
        node_list_v[i] = {}
        node_list_v[i]['embedding_vectors'] = preprocessing.normalize(vectors, norm='l2')
        node_list_v[i]['context_vectors'] = preprocessing.normalize(help_vectors, norm='l2')

In [50]:
## 初始化向量embedding
node_list_u, node_list_v = {}, {}
init_embedding_vectors(node_u, node_v, node_list_u, node_list_v, args)
last_loss, count, epsilon = 0, 0, 1e-3

getting context and negative samples....
negative samples is ok.....
context...
context...ok
context...
context...ok


## 8. 正式开始训练

In [51]:
import math
def skip_gram(center, contexts, negs, node_list, lam, pa):
    """
    skip-gram
    :param center:
    :param contexts:
    :param negs:
    :param node_list:
    :param lam:
    :param pa:
    :return:
    """
    loss = 0
    ##创建一个指示函数 I_z，用于标记中心词（值为 1）
    I_z = {center: 1}  # indication function
    ## 对于负样本节点
    for node in negs:
        ## 负样本词节点添加到指示函数 I_z 中，并设置其值为 0
        I_z[node] = 0
    ## 获取上下文节点嵌入向量    
    V = np.array(node_list[contexts]['embedding_vectors'])
    ## 初始化更新向量
    update = [[0] * V.size]
    
    ##遍历所有节点
    for u in I_z.keys():
        if node_list.get(u) is  None:
            pass
        ##获取节点的上下文向量 Theta
        Theta = np.array(node_list[u]['context_vectors'])
        ##计算嵌入向量 V 和上下文向量 Theta 的点积 X
        X = float(V.dot(Theta.T))
        ##并计算 sigmoid 函数值
        sigmod = 1.0 / (1 + (math.exp(-X * 1.0)))
        ##计算更新向量和上下文向量的更新
        update += pa * lam * (I_z[u] - sigmod) * Theta
        node_list[u]['context_vectors'] += pa * lam * (I_z[u] - sigmod) * V
        
        #尝试更新损失函数，损失函数基于对数似然
        try:
            loss += pa * (I_z[u] * math.log(sigmod) + (1 - I_z[u]) * math.log(1 - sigmod))
        except:
            pass
            # print "skip_gram:",
            # print(V,Theta,sigmod,X,math.exp(-X * 1.0),round(math.exp(-X * 1.0),10))
    return update, loss


In [52]:
## 这个函数用于在推荐系统或类似的应用中更新用户和项目的嵌入向量。它根据两个节点间的相互作用（通过边权重 e_ij 表示）以及它们当前的嵌入状态，使用 KL 散度作为损失函数来优化嵌入向量。
def KL_divergence(edge_dict_u, u, v, node_list_u, node_list_v, lam, gamma):
    """
    KL-divergenceO1
    :param edge_dict_u:
    :param u:
    :param v:
    :param node_list_u:
    :param node_list_v:
    :param lam:
    :param gamma:
    :return:
    """
    loss = 0
    e_ij = edge_dict_u[u][v]

    update_u = 0
    update_v = 0
    ## 获取嵌入向量
    U = np.array(node_list_u[u]['embedding_vectors'])
    V = np.array(node_list_v[v]['embedding_vectors'])
    
## 显性关系
    ##计算u节点和v节点之间的点积
    X = float(U.dot(V.T))
    ##通过sigmoid得到其之间的相似性
    sigmod = 1.0 / (1 + (math.exp(-X * 1.0)))
    ## 根据sigmod函更新用户 u 和项目 v 的嵌入向量
    update_u += gamma * lam * ((e_ij * (1 - sigmod)) * 1.0 / math.log(math.e, math.e)) * V
    update_v += gamma * lam * ((e_ij * (1 - sigmod)) * 1.0 / math.log(math.e, math.e)) * U

    try:
        loss += gamma * e_ij * math.log(sigmod)
    except:
        pass
        # print "KL:",
        # print(U,V,sigmod,X,math.exp(-X * 1.0),round(math.exp(-X * 1.0),10))
    return update_u, update_v, loss


In [53]:
def ndarray_tostring(array):
    string = ""
    for item in array[0]:
        string += str(item).strip()+" "
    return string+"\n"

In [54]:
def save_to_file(node_list_u,node_list_v,model_path,args):
    with open(args.vectors_u,"w") as fw_u:
        for u in node_list_u.keys():
            fw_u.write(u+" "+ ndarray_tostring(node_list_u[u]['embedding_vectors']))
    with open(args.vectors_v,"w") as fw_v:
        for v in node_list_v.keys():
            fw_v.write(v+" "+ndarray_tostring(node_list_v[v]['embedding_vectors']))

In [55]:
import sys
print("============== training ==============")
for iter in range(0, args.max_iter):
    ## 初始化
    s1 = "\r[%s%s]%0.2f%%"%("*"* iter," "*(args.max_iter-iter),iter*100.0/(args.max_iter-1))
    loss = 0
    visited_u = dict(zip(node_list_u.keys(), [0] * len(node_list_u.keys())))
    visited_v = dict(zip(node_list_v.keys(), [0] * len(node_list_v.keys())))
    random.shuffle(edge_list)
    ## 隐式关系捕捉
    for i in range(len(edge_list)):
        u, v, w = edge_list[i]
        ##获取用u节点的上下文长度，并随机打乱这个上下文列表
        length = len(context_dict_u[u])
        random.shuffle(context_dict_u[u])
        if visited_u.get(u) < length:
            #创建一个索引列表 index_list，从 visited_u[u] 开始，直到 visited_u[u] + 1 或上下文长度的最小值
            index_list = list(range(visited_u.get(u),min(visited_u.get(u)+1,length)))
            #遍历索引列表，对每个索引，获取对应的上下文 context_u 和负样本 neg_u
            for index in index_list:
                context_u = context_dict_u[u][index]
                neg_u = neg_dict_u[u][index]
                # center,context,neg,node_list,eta
    ##8-1 skip_gram 对于上下文中的每个节点 z，调用 skip_gram 函数进行节点u的嵌入更新
                for z in context_u:
    
                    tmp_z, tmp_loss = skip_gram(u, z, neg_u, node_list_u, lam, alpha)
                    node_list_u[z]['embedding_vectors'] += tmp_z
                    loss += tmp_loss
            visited_u[u] = index_list[-1]+3
            
            
        ##同理在节点v上进行
        length = len(context_dict_v[v])
        random.shuffle(context_dict_v[v])
        if visited_v.get(v) < length:
            # print(v)
            index_list = list(range(visited_v.get(v),min(visited_v.get(v)+1,length)))
            for index in index_list:
                context_v = context_dict_v[v][index]
                neg_v = neg_dict_v[v][index]
                # center,context,neg,node_list,eta
                for z in context_v:
                    tmp_z, tmp_loss = skip_gram(v, z, neg_v, node_list_v, lam, beta)
                    node_list_v[z]['embedding_vectors'] += tmp_z
                    loss += tmp_loss
            visited_v[v] = index_list[-1]+3
        
        ## 8-2显示关系捕捉利用KL_divergence来进行嵌入更新
        update_u, update_v, tmp_loss = KL_divergence(edge_dict_u, u, v, node_list_u, node_list_v, lam, gamma)
        loss += tmp_loss
        node_list_u[u]['embedding_vectors'] += update_u
        node_list_v[v]['embedding_vectors'] += update_v

    delta_loss = abs(loss - last_loss)
    if last_loss > loss:
        lam *= 1.05
    else:
        lam *= 0.95
    last_loss = loss
    if delta_loss < epsilon:
        break
    sys.stdout.write(s1)
    sys.stdout.flush()
save_to_file(node_list_u,node_list_v,model_path,args)
print("")


============== training ==============


C:\Users\Wei Zhou\AppData\Local\Temp\ipykernel_22160\529322566.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X = float(V.dot(Theta.T))
C:\Users\Wei Zhou\AppData\Local\Temp\ipykernel_22160\1414161090.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X = float(U.dot(V.T))


[************************************************* ]100.00%


## 测试

In [56]:
def nDCG(ranked_list, ground_truth):
    dcg = 0
    idcg = IDCG(len(ground_truth))
    for i in range(len(ranked_list)):
        id = ranked_list[i]
        if id not in ground_truth:
            continue
        rank = i+1
        dcg += 1/ math.log(rank+1, 2)
    return dcg / idcg

def IDCG(n):
    idcg = 0
    for i in range(n):
        idcg += 1 / math.log(i+2, 2)
    return idcg

def AP(ranked_list, ground_truth):
    hits, sum_precs = 0, 0.0
    for i in range(len(ranked_list)):
        id = ranked_list[i]
        if id in ground_truth:
            hits += 1
            sum_precs += hits / (i+1.0)
    if hits > 0:
        return sum_precs / len(ground_truth)
    else:
        return 0.0

def RR(ranked_list, ground_list):

    for i in range(len(ranked_list)):
        id = ranked_list[i]
        if id in ground_list:
            return 1 / (i + 1.0)
    return 0

def precision_and_recall(ranked_list,ground_list):
    hits = 0
    for i in range(len(ranked_list)):
        id = ranked_list[i]
        if id in ground_list:
            hits += 1
    pre = hits/(1.0 * len(ranked_list))
    rec = hits/(1.0 * len(ground_list))
    return pre, rec

In [57]:
def top_N(test_u, test_v, test_rate, node_list_u, node_list_v, top_n):
    recommend_dict = {}
    for u in test_u:
        recommend_dict[u] = {}
        for v in test_v:
            if node_list_u.get(u) is None:
                pre = 0
            else:
                U = np.array(node_list_u[u]['embedding_vectors'])
                if node_list_v.get(v) is None:
                    pre = 0
                else:
                    V = np.array(node_list_v[v]['embedding_vectors'])
                    pre = U.dot(V.T)[0][0]
            recommend_dict[u][v] = float(pre)

    precision_list = []
    recall_list = []
    ap_list = []
    ndcg_list = []
    rr_list = []

    for u in test_u:
        tmp_r = sorted(recommend_dict[u].items(), key=lambda x: x[1], reverse=True)[0:min(len(recommend_dict[u]), top_n)]
        tmp_t = sorted(test_rate[u].items(), key=lambda x: x[1], reverse=True)[0:min(len(test_rate[u]), top_n)]
        tmp_r_list = []
        tmp_t_list = []
        for (item, rate) in tmp_r:
            tmp_r_list.append(item)

        for (item, rate) in tmp_t:
            tmp_t_list.append(item)
        pre, rec = precision_and_recall(tmp_r_list,tmp_t_list)
        ap = AP(tmp_r_list,tmp_t_list)
        rr = RR(tmp_r_list,tmp_t_list)
        ndcg = nDCG(tmp_r_list,tmp_t_list)
        precision_list.append(pre)
        recall_list.append(rec)
        ap_list.append(ap)
        rr_list.append(rr)
        ndcg_list.append(ndcg)
    precison = sum(precision_list) / len(precision_list)
    recall = sum(recall_list) / len(recall_list)
    #print(precison, recall)
    f1 = 2 * precison * recall / (precison + recall)
    map = sum(ap_list) / len(ap_list)
    mrr = sum(rr_list) / len(rr_list)
    mndcg = sum(ndcg_list) / len(ndcg_list)
    return f1,map,mrr,mndcg

In [58]:
from sklearn.metrics import average_precision_score
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
def generateFeatureFile(filecase,filevector_u,filevector_v,fileout,factors):
    vectors_u = {}
    vectors_v = {}
    with open(filevector_u,'r') as fu:
        for line in fu.readlines():
            items = line.strip().split(' ')
            vectors_u[items[0]] = items[1:]
    with open(filevector_v,'r') as fv:
        for line in fv.readlines():
            items = line.strip().split(' ')
            vectors_v[items[0]] = items[1:]
    with open(filecase,'r') as fc, open(fileout,'w') as fo:
        for line in fc.readlines():
            items = line.strip().split('\t')
            if vectors_u.get(items[0]) == None:
                vectors_u[items[0]] = ['0'] * factors
            if vectors_v.get(items[1]) == None:
                vectors_v[items[1]] = ['0'] * factors
            if items[-1] == '1':
                fo.write('{}\t{}\t{}\n'.format('\t'.join(vectors_u[items[0]]),'\t'.join(vectors_v[items[1]]),1))
            else:
                fo.write('{}\t{}\t{}\n'.format('\t'.join(vectors_u[items[0]]),'\t'.join(vectors_v[items[1]]),0))


In [59]:
def link_prediction(args):
    filecase_a = args.case_train
    filecase_e = args.case_test
    filevector_u = args.vectors_u
    filevector_v = args.vectors_v
    filecase_a_c = r'../data/features_train.dat'
    filecase_e_c = r'../data/features_test.dat'
    generateFeatureFile(filecase_a,filevector_u,filevector_v,filecase_a_c,args.d)
    generateFeatureFile(filecase_e,filevector_u,filevector_v,filecase_e_c,args.d)

    df_data_train = pd.read_csv(filecase_a_c,header = None,sep='\t',encoding='utf-8')
    X_train = df_data_train.drop(len(df_data_train.keys())-1,axis = 1)
    y_train = df_data_train[len(df_data_train.keys())-1]

    df_data_test = pd.read_csv(filecase_e_c,header = None,sep='\t',encoding='utf-8')
    X_test = df_data_test.drop(len(df_data_train.keys())-1,axis = 1)
    X_test = X_test.fillna(X_test.mean())
    y_test = df_data_test[len(df_data_test.keys())-1]
    y_test_list = list(y_test)

    lg = LogisticRegression(penalty='l2',C=0.001)
    lg.fit(X_train,y_train)
    lg_y_pred_est = lg.predict_proba(X_test)[:,1]
    fpr,tpr,thresholds = metrics.roc_curve(y_test,lg_y_pred_est)
    average_precision = average_precision_score(y_test, lg_y_pred_est)
    os.remove(filecase_a_c)
    os.remove(filecase_e_c)
    return metrics.auc(fpr,tpr), average_precision

In [65]:
if args.rec:
    print("============== testing ===============")
    f1, map, mrr, mndcg = top_N(test_user,test_item,test_rate,node_list_u,node_list_v,args.top_n)
    print('recommendation metrics: F1 : %0.4f, MAP : %0.4f, MRR : %0.4f, NDCG : %0.4f' % (round(f1,4), round(map,4), round(mrr,4), round(mndcg,4)))
if args.lip:
    print("============== testing ===============")
    auc_roc, auc_pr = link_prediction(args)
    print('link prediction metrics: AUC_ROC : %0.4f, AUC_PR : %0.4f' % (round(auc_roc,4), round(auc_pr,4)))


============== testing ===============
link prediction metrics: AUC_ROC : 0.9109, AUC_PR : 0.9415
